In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
from tkinter import font
from turtle import color
from cairo import FontWeight
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

#Loading data
df = pd.read_csv('booking_list_in_vilnius_review_more_than_10.csv', encoding='utf-8')

#Separating date interval
split_dates = df['date'].str.split('-', n=3, expand=True)
df['start_date'] = split_dates[0] + '-' + split_dates[1] + '-' + split_dates[2]
df['end_date'] = split_dates[3].str[:4] + '-' + split_dates[3].str[5:7] + '-' + split_dates[3].str[8:]


#Converting dates to a datetime format
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

#extracting month from 'start date'
df['start_month'] = df['start_date'].dt.month


#creating binary columns for each month from 1 to 12.
for month in range(1, 13):
    df[f'start_month_{month}'] = df['start_month'] == month


#converting price column to numeric value, removing eur symbol and commas
df['price'] = df['price'].replace('[\€,]', '', regex=True)
df['price'] = df['price'].str.replace(' ', '') #removing spaces
df['price'] = pd.to_numeric(df['price'], errors='coerce') #converting to numeric

#convert review quantity to numeric
df['review_quantity'] = pd.to_numeric(df['review_quantity'], errors='coerce')

# Converting score to numeric
df['score'] = df['score'].str.extract(r'(\d+\,\d+)')[0]  # Extracting numeric part with comma
df['score'] = df['score'].str.replace(',', '.').astype(float)  # Replacing comma with dot and converting to float


In [3]:

results = {}
min_prices = []
max_prices = []
mean_prices = []

for month in range(1, 13):
    month_column = f'start_month_{month}'

    #filtered dataframe for records where the current month column is True
    month_data = df[df[month_column]]

    #cheking is the are any data for this month
    if not month_data.empty:
        min_price = month_data['price'].min()
        max_price = month_data['price'].max()
        mean_price = round(month_data['price'].mean(), 2)

        min_prices.append(min_price)
        max_prices.append(max_price)
        mean_prices.append(mean_price)

        #sort by review quantity as MOST POPULAR
        month_data_sorted_by_reviews = month_data.sort_values(by=['review_quantity', 'score'], ascending=False)

        #Find 10 cheapest results
        top_10_cheapest = month_data_sorted_by_reviews.nsmallest(10, 'price')

        #Find 10 most expensive results
        top_10_most_expensive = month_data_sorted_by_reviews.nlargest(10, 'price')

        results[f'Month_{month}'] = {
            'Min Price': min_price,
            'Max Price': max_price,
            'Mean Price': mean_price, 
            'Top 10 Cheapest': top_10_cheapest, 
            'Top 10 Most Expensive': top_10_most_expensive
        }
    else:
        results[f'Month_{month}'] = {
            'Min Price': None,
            'Max Price': None,
            'Mean Price': None,
            'Top 10 Cheapest': pd.DataFrame(), 
            'Top 10 Most Expensive': pd.DataFrame()
        }



# Convert min_prices, max_prices, and mean_prices to pandas Series for plotting
months = list(range(1, 13))
min_prices = pd.Series(min_prices, index=months)
max_prices = pd.Series(max_prices, index=months)
mean_prices = pd.Series(mean_prices, index=months)

In [4]:
# Droping all unnecessary columns

df = df.iloc[:,:8]
df

,title,review_quantity,price,score,date,link,start_date,end_date
0,COZY apartment in the City Center,18,80,9.7,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/cozy-apartmen...,2024-09-01,2024-09-02
1,"HaPPy Inn GOLD Studio, self check-in, Parking ...",104,84,9.9,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/happy-inn-gol...,2024-09-01,2024-09-02
2,Lovely apartment Traku str.,112,86,9.8,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/lovely-apartm...,2024-09-01,2024-09-02
3,The Joseph,635,95,9.5,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/the-joseph.lt...,2024-09-01,2024-09-02
4,"Angel House Vilnius, 36A",11,98,NaN,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/angel-house-v...,2024-09-01,2024-09-02
...,...,...,...,...,...,...,...,...
10827,Luxurious apartment,168,74,9.7,2025-08-01-2025-08-02,https://www.booking.com/hotel/lt/luxurious-apa...,2025-08-01,2025-08-02
10828,German18-3A Luxury Vilnius apartment,141,355,9.9,2025-08-01-2025-08-02,https://www.booking.com/hotel/lt/vokieciu-18-3...,2025-08-01,2025-08-02
10829,Tyzen38 Luxury Apartment,49,144,9.6,2025-08-01-2025-08-02,https://www.booking.com/hotel/lt/tyzenhauz38-l...,2025-08-01,2025-08-02
10830,German18-3B Luxury Vilnius apartment,170,323,9.8,2025-08-01-2025-08-02,https://www.booking.com/hotel/lt/german-18-3b-...,2025-08-01,2025-08-02


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10832 entries, 0 to 10831
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            10832 non-null  object        
 1   review_quantity  10832 non-null  int64         
 2   price            10832 non-null  int64         
 3   score            9768 non-null   float64       
 4   date             10832 non-null  object        
 5   link             10832 non-null  object        
 6   start_date       10832 non-null  datetime64[ns]
 7   end_date         10832 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 677.1+ KB


In [6]:
df[df['score'].isnull()]

,title,review_quantity,price,score,date,link,start_date,end_date
4,"Angel House Vilnius, 36A",11,98,NaN,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/angel-house-v...,2024-09-01,2024-09-02
18,"HaPPy Inn Ugnė, The studio is in the heart of ...",37,74,NaN,2024-09-01-2024-09-02,https://www.booking.com/hotel/lt/happy-inn-ugn...,2024-09-01,2024-09-02
59,"HaPPy Inn Ugnė, The studio is in the heart of ...",37,78,NaN,2024-09-02-2024-09-03,https://www.booking.com/hotel/lt/happy-inn-ugn...,2024-09-02,2024-09-03
102,"HaPPy Inn Ugnė, The studio is in the heart of ...",37,82,NaN,2024-09-03-2024-09-04,https://www.booking.com/hotel/lt/happy-inn-ugn...,2024-09-03,2024-09-04
150,"HaPPy Inn Ugnė, The studio is in the heart of ...",37,82,NaN,2024-09-04-2024-09-05,https://www.booking.com/hotel/lt/happy-inn-ugn...,2024-09-04,2024-09-05
...,...,...,...,...,...,...,...,...
10799,Sweet home30,63,60,NaN,2025-07-30-2025-07-31,https://www.booking.com/hotel/lt/sweet-home2.l...,2025-07-30,2025-07-31
10809,"Angel House Vilnius, 36A",11,159,NaN,2025-07-31-2025-08-01,https://www.booking.com/hotel/lt/angel-house-v...,2025-07-31,2025-08-01
10812,Sweet home30,63,60,NaN,2025-07-31-2025-08-01,https://www.booking.com/hotel/lt/sweet-home2.l...,2025-07-31,2025-08-01
10822,"Angel House Vilnius, 36A",11,164,NaN,2025-08-01-2025-08-02,https://www.booking.com/hotel/lt/angel-house-v...,2025-08-01,2025-08-02


In [7]:
# Mean Inputation
df['score'] = df['score'].fillna(df['score'].mean())


In [8]:
df_prepared = df.drop(columns=['link', 'title', 'date', 'end_date'])


In [9]:
df_prepared

,review_quantity,price,score,start_date
0,18,80,9.700000,2024-09-01
1,104,84,9.900000,2024-09-01
2,112,86,9.800000,2024-09-01
3,635,95,9.500000,2024-09-01
4,11,98,9.695321,2024-09-01
...,...,...,...,...
10827,168,74,9.700000,2025-08-01
10828,141,355,9.900000,2025-08-01
10829,49,144,9.600000,2025-08-01
10830,170,323,9.800000,2025-08-01


In [10]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10832 entries, 0 to 10831
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   review_quantity  10832 non-null  int64         
 1   price            10832 non-null  int64         
 2   score            10832 non-null  float64       
 3   start_date       10832 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 338.6 KB


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

In [12]:
#Spliting start date column to 3 different columns
df_prepared['year'] = df_prepared['start_date'].dt.year
df_prepared['month'] = df_prepared['start_date'].dt.month
df_prepared['day'] = df_prepared['start_date'].dt.day

#Handling datetime columns
datetime_columns = df_prepared.select_dtypes(include=[np.datetime64]).columns

# # Convert datetime columns to numerical features
# for col in datetime_columns:
#     df_prepared[col + '_year'] = df_prepared[col].dt.year
#     df_prepared[col + '_month'] = df_prepared[col].dt.month
#     df_prepared[col + '_day'] = df_prepared[col].dt.day

#     # Drop the original datetime column
#     df_prepared = df_prepared.drop(columns=[col])

# Drop 'start_date' column if it still exists
df_prepared = df_prepared.drop(columns=['start_date'], errors='ignore')

# Save prepared DataFrame to a CSV with space between text
df_prepared.to_csv('df_prepared.csv', index=False, na_rep='NA')

In [13]:
df_prepared = pd.read_csv('df_prepared.csv')

X = df_prepared.drop(columns=['price'])
y = df_prepared['price']